In [2]:
import logging
import warnings
import joblib

import numpy as np
import pandas as pd
import geopandas as gpd
from sklearn.neighbors import NearestNeighbors

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
warnings.resetwarnings()
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option("display.max_columns", None)

In [5]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(levelname)s | %(funcName)s - %(message)s', datefmt='%I:%M:%S')
logger = logging.getLogger(__name__)

### Preprocessing: Climate solution counts per domain

In [ ]:
abstract_solutions = pd.read_parquet('../cities-learning-dec/data/climate_solutions_typology/oa_sentence_solutions_2.parquet')
abstract_city_mapping = pd.read_csv('../cities-learning-dec/data/geoparser/clean_places_augmented_dedup.csv', index_col=0)

abstract_solutions = pd.merge(abstract_solutions, abstract_city_mapping[['id', 'city_id']], on='id')

domain_to_solution_ids = {
    "Mobility": [1, 2, 3, 4, 5, 6, 7],
    "Buildings": [8, 9, 10, 11, 12, 13, 14, 15, 16],
    "Energy": [17, 18, 19, 20, 21, 22, 23],
    "Thermal comfort and Heat stress management": [24, 25, 26, 27],
    "Food provisioning systems": [28, 29, 30, 31, 32, 33, 34],
    "Water": [35, 36, 37, 38, 39, 40],
    "Waste management": [41, 42, 43, 44],
    "Disaster and risk management": [45, 46, 47, 48, 49, 50],
    "Carbon dioxide removal": [51, 52, 53, 54],
}

for domain, solution_ids in domain_to_solution_ids.items():
    abstract_solutions[domain] = abstract_solutions[[f'solution_{i}_match' for i in solution_ids]].max(axis=1)

domains = list(domain_to_solution_ids.keys())
abstract_solution_counts = abstract_solutions.groupby(['city_id','id'])[domains].max()
city_solution_counts = abstract_solution_counts.groupby('city_id')[domains].sum()

city_solution = pd.DataFrame({
    'GHS_urban_area_id': city_solution_counts.index,
    'n_solutions': city_solution_counts.sum(axis=1),
    'solution_domain_counts': city_solution_counts.to_dict(orient="records"),
})


### Preprocessing: Nearest neighbors in embedding space

In [13]:
# Workaround for numpy version compatibility issues

import sys
import types
import numpy as np

# Create fake parent module
np__core = types.ModuleType("numpy._core")

# Map numpy._core.multiarray -> numpy.core.multiarray
np__core.multiarray = np.core.multiarray

# Inject modules into sys.modules
sys.modules["numpy._core"] = np__core
sys.modules["numpy._core.multiarray"] = np__core.multiarray


In [14]:
nn_all_runs = []
for i in range(30):
    embeddings = joblib.load(f'../cities-learning-dec/clustering_models/latent_representation/latent_run_{i}.pkl')

    emb_cols = ["latent_0", "latent_1", "latent_2", "latent_3"]
    X = embeddings[emb_cols].values

    nn = NearestNeighbors(n_neighbors=51, metric="euclidean")
    nn.fit(X)

    distances, indices = nn.kneighbors(X)

    # Remove self-index (first column)
    neighbor_indices = indices[:, 1:]
    neighbor_distances = distances[:, 1:]

    city_ids = embeddings["GHS_urban_area_id"].astype(int).values


    N, K = neighbor_indices.shape

    # Repeat each city ID K times → shape (N*K,)
    city_a = np.repeat(city_ids, K)

    # Flatten neighbors and distances (already in the correct order)
    city_b = city_ids[neighbor_indices].ravel()
    dist = neighbor_distances.ravel()

    nn_long = pd.DataFrame({
        "city_a": city_a,
        "city_b": city_b,
        "distance": dist
    })
    nn_all_runs.append(nn_long)

nn_long = pd.concat(nn_all_runs)

nn = (
    nn_long.groupby(['city_a', 'city_b'], as_index=False)
          .distance.mean()
)

nn_top20 = (
    nn.sort_values(['city_a', 'distance'])
     .groupby('city_a')
     .head(20)
     .reset_index(drop=True)
)

nn_top20['distance'] = nn_top20['distance'].round(4)
nn_top20['distance'] = nn_top20['distance'].round(4)

nn_wide = (
    nn_top20
    .groupby("city_a")
    .agg({
        "city_b": list,
        "distance": list
    })
    .reset_index()
)

nn_wide = nn_wide.rename(columns={
    "city_a": "GHS_urban_area_id",
    "city_b": "neighbors",
    "distance": "neighbor_distances"
})

### Merging all data sources

In [20]:
city_meta = pd.read_csv('../cities-learning-dec/data/clustering_results/cities_by_regional_type_clean.csv', index_col=0).rename(columns={'city_id': 'GHS_urban_area_id'})[['GHS_urban_area_id', 'city_name', 'country', 'Region', 'n_studies']]
city_characteristics = pd.read_parquet('../cities-learning-dec/data/clustering_data_clean/GHS_UCDB_2024_preproc_2025_04_09_uci_and_nan_imputation_add_vars_included.parquet')
embeddings = joblib.load('../cities-learning-dec/clustering_models/latent_representation/latent_run_0.pkl')
city_socioeconomics = pd.read_csv('../cities-learning-dec/data/GHS_UCDB_GLOBE_R2024A_V1_0/socioeconomic.csv')[['ID_UC_G0', 'SC_SEC_GDF_2020', 'SC_SEC_HDI_2020']].rename(columns={'ID_UC_G0': 'GHS_urban_area_id', 'SC_SEC_GDF_2020': 'GHS_female_gender_index', 'SC_SEC_HDI_2020': 'GHS_HDI'})

city_assignment_proba = pd.read_csv('../cities-learning-dec/data/clustering_results/dec_clusters_k4.csv')
city_assignment_proba['probability'] = city_assignment_proba[['mean_prob_cluster_0', 'mean_prob_cluster_1', 'mean_prob_cluster_2', 'mean_prob_cluster_3']].max(axis=1)
mixed_thresholds = city_assignment_proba.groupby('consensus_label_majority')['probability'].median() * 0.65
city_assignment_proba['mixed'] = city_assignment_proba['probability'] < city_assignment_proba['consensus_label_majority'].map(mixed_thresholds)
city_assignment_proba['type'] = city_assignment_proba['consensus_label_majority'].map({0: "Type 2", 1: "Type 3", 2: "Type 1", 3: "Type 4"})
city_assignment_proba.loc[city_assignment_proba['mixed'], 'type'] = "Mixed"

emissions = pd.read_csv('../cities-learning-dec/data/emissions/balance_sheet.csv')
emissions = emissions[emissions['Year'] == 2022][['ID_UC_G0', 'ODIAC']].rename(columns={'ID_UC_G0': 'GHS_urban_area_id', 'ODIAC': 'total_emissions'})

geometries = gpd.read_parquet('../cities-learning-dec/data/clustering_data_clean/GHS_UCDB_2024_preproc_2025_04_03.parquet', columns=['GHS_urban_area_id', 'geometry'])
geometries['lat'] = geometries.centroid.to_crs(epsg=4326).y
geometries['lon'] = geometries.centroid.to_crs(epsg=4326).x
geometries = geometries[['GHS_urban_area_id', 'lat', 'lon']]

cities = pd.merge(city_characteristics, city_meta, on='GHS_urban_area_id')
cities = pd.merge(city_assignment_proba, cities, on='GHS_urban_area_id', how='left')
cities = pd.merge(cities, embeddings, on='GHS_urban_area_id', how='left')
cities = pd.merge(cities, nn_wide, on='GHS_urban_area_id', how='left')
cities = pd.merge(cities, city_socioeconomics, on='GHS_urban_area_id', how='left')
cities = pd.merge(cities, geometries, on='GHS_urban_area_id', how='left')
cities = pd.merge(cities, city_solution, on='GHS_urban_area_id', how='left')
cities = pd.merge(cities, emissions, on='GHS_urban_area_id', how='left')

cities['emissions'] = cities['total_emissions'] / cities['GHS_population']
cities['city_name'].fillna('Unknown', inplace=True)
cities['embedding'] = cities[['latent_0', 'latent_1', 'latent_2', 'latent_3']].to_numpy().tolist()
# cities['type_probabilities'] = cities[['mean_prob_cluster_1', 'mean_prob_cluster_2', 'mean_prob_cluster_3', 'mean_prob_cluster_0']].to_numpy().tolist()
cities['type_probabilities'] = cities[['mean_prob_cluster_2', 'mean_prob_cluster_0', 'mean_prob_cluster_1', 'mean_prob_cluster_3']].to_numpy().tolist()

In [16]:
mixed_thresholds

consensus_label_majority
0    0.442
1    0.520
2    0.533
3    0.286
Name: probability, dtype: float64

### Rename

In [21]:
cities = cities.rename(columns={
    'GHS_urban_area_id': 'id',
    'city_name': 'name',
    'Region': 'region',
    'GHS_population': 'population',
    'GHS_population_growth': 'population_growth',
    'GHS_population_density': 'population_density',
    'GHS_population_density_growth': 'population_density_growth',
    'GHS_GDP_PPP': 'gdp_ppp',
    'GHS_GDP_PPP_growth': 'gdp_ppp_growth',
    'GHS_critical_infra': 'critical_infrastructure',
    'GHS_greenness_index': 'greenness_index',
    'GHS_precipitation': 'precipitation',
    'GHS_HDI': 'hdi',
    'GHS_female_gender_index': 'female_gender_index',
})


### Compute metric ranks and store as JSON

In [22]:
info_cols = [
    'id',
    'name',
    'country',
    'region',
    'type',
    'n_studies',
    'n_solutions',
    'solution_domain_counts',
    'probability',
    'embedding',
    'neighbors',
    'neighbor_distances',
    'type_probabilities',
    'lat',
    'lon',
]

metric_cols = [
    'population',
    'population_growth',
    'population_density',
    'population_density_growth',
    'gdp_ppp',
    'gdp_ppp_growth',
    'critical_infrastructure',
    'greenness_index',
    'precipitation',
    'hdd',
    'cdd',
    'hdi',
    'female_gender_index',
    'emissions',
]

for col in metric_cols:
    cities[f"{col}_pct"] = (cities[col].rank(pct=True) * 100).round()

pct_cols = [f"{col}_pct" for col in metric_cols]

cities[info_cols + metric_cols + pct_cols].to_json('public/cities.json', orient='records')

In [117]:
cities.groupby('type')['region'].value_counts(normalize=True).round(2)

type    region       
Mixed   Asia             0.43
        Africa           0.30
        South America    0.16
        North America    0.04
        Europe           0.03
        Small Islands    0.03
        Australasia      0.00
Type 1  Asia             0.63
        Africa           0.31
        South America    0.04
        Small Islands    0.02
        North America    0.00
        Europe           0.00
Type 2  Asia             0.69
        South America    0.12
        Africa           0.08
        Europe           0.06
        North America    0.04
        Small Islands    0.01
Type 3  Europe           0.51
        Asia             0.25
        North America    0.19
        South America    0.02
        Australasia      0.02
        Small Islands    0.01
Type 4  Asia             0.60
        Africa           0.28
        South America    0.05
        Europe           0.04
        North America    0.03
Name: region, dtype: float64

In [118]:
geometries.to_csv('cities-lat-lon.csv', index=False)